# Sistem Pengecekan Manual Labeling

In [69]:
import pandas as pd
import numpy as np
import os
from IPython.display import display, clear_output
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [70]:
df = pd.read_csv('yelp_labeled_full.csv')

In [71]:
df['is_fake'].value_counts()

is_fake
0    6521
1    1151
Name: count, dtype: int64

In [14]:
selected_columns = [
    'business_name', 'review_id', 'user_id', 'date', 'stars_review',
    'stars_business_average', 'normalized_text', 'lemmatized_ml',
    'lemmatized_dl', 'lemmatized_bert', 'sentiment_label', 'daily_spike_score',
    'hourly_spike_score', 'temporal_spike_score', 'user_review_count',
    'avg_user_rating', 'review_frequency', 'suspicious_user',
    'user_behavior_score', 'text_similarity_score',
    'rating_manipulation_score', 'consistency_score', 'fake_probability', 'is_fake',
    'fake_explanation'
]

df_check = df[selected_columns].copy()

print(f"✅ Data siap untuk dicek! Total {len(df_check)} baris")

print("🔀 Membagi data menjadi dua bagian untuk dua pengecek...")
total_rows = len(df_check)
split_index = total_rows // 2

df_A = df_check.iloc[:split_index].copy()
df_B = df_check.iloc[split_index:].copy()

print(f"📊 Pembagian data:")
print(f"Data A: {len(df_A)} baris (indeks 0-{split_index-1})")
print(f"Data B: {len(df_B)} baris (indeks {split_index}-{total_rows-1})")

✅ Data siap untuk dicek! Total 7672 baris
🔀 Membagi data menjadi dua bagian untuk dua pengecek...
📊 Pembagian data:
Data A: 3836 baris (indeks 0-3835)
Data B: 3836 baris (indeks 3836-7671)


In [15]:
df_A.to_excel("yelp_labeled_part_A.xlsx", index=False)
df_B.to_excel("yelp_labeled_part_B.xlsx", index=False)

print("💾 Data berhasil disimpan sebagai file Excel:")
print("   - yelp_labeled_part_A.xlsx")
print("   - yelp_labeled_part_B.xlsx")

💾 Data berhasil disimpan sebagai file Excel:
   - yelp_labeled_part_A.xlsx
   - yelp_labeled_part_B.xlsx


## Kalo dah di Check

In [72]:
# Baca dua file hasil revisi
df_A_new = pd.read_excel("yelp_labeled_part_A.xlsx")
df_B_new = pd.read_excel("yelp_labeled_part_B.xlsx")

# Gabungkan kembali
df_merged = pd.concat([df_A_new, df_B_new], ignore_index=True)

print("✅ Data berhasil digabung kembali sesuai urutan semula!")
print(df_merged.head())

df_merged.to_csv("yelp_labeled_recombined.csv", index=False)

✅ Data berhasil digabung kembali sesuai urutan semula!
       business_name               review_id                 user_id  \
0  Acme Oyster House  qdpoNaAd2d8hz354jRKryg  M2jgPvf6syoOHGW0R8PotQ   
1  Acme Oyster House  5t14YNj5mji1aoIbt3uoQQ  akv8uZ1X6POqCWXYcnbXfA   
2  Acme Oyster House  b2lwJxRgseknM_0B5cRkFQ  2zhTsvYOEPQaQP9w53zkvA   
3  Acme Oyster House  _BAe5c9nCJh7ntBo2YMqsQ  eZGoPMRfq2hw5LfwJg0uTA   
4  Acme Oyster House  rFeHUzOELJlba9vHxvR6Ug  uBvrdhNSlGnN35NA8cSMwQ   

                  date  stars_review  stars_business_average  \
0  2012-02-07 00:05:19             2                       4   
1  2015-02-24 01:49:26             4                       4   
2  2010-01-07 18:09:30             2                       4   
3  2010-01-07 18:12:23             3                       4   
4  2010-01-11 02:27:03             4                       4   

                                                                                                                               

In [73]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7670 entries, 0 to 7669
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   business_name              7670 non-null   object 
 1   review_id                  7670 non-null   object 
 2   user_id                    7670 non-null   object 
 3   date                       7670 non-null   object 
 4   stars_review               7670 non-null   int64  
 5   stars_business_average     7670 non-null   int64  
 6   normalized_text            7670 non-null   object 
 7   lemmatized_ml              7670 non-null   object 
 8   lemmatized_dl              7670 non-null   object 
 9   lemmatized_bert            7670 non-null   object 
 10  sentiment_label            7670 non-null   object 
 11  daily_spike_score          7670 non-null   float64
 12  hourly_spike_score         7670 non-null   float64
 13  temporal_spike_score       7670 non-null   float

In [74]:
sentiment_counts = df_merged['sentiment_label'].value_counts()
sentiment_counts

sentiment_label
positive    6263
negative    1069
neutral      338
Name: count, dtype: int64

In [75]:
total_reviews = len(df_merged)
total_reviews

7670

In [76]:
print("AFTER CHECK:")
print("📊 SENTIMENT ANALYSIS RESULTS:")
print("=" * 50)

for sentiment in ['positive', 'negative', 'neutral']:
    count = sentiment_counts.get(sentiment, 0)
    percent = (count / total_reviews) * 100
    print(f"   {sentiment.upper():<9}: {count} reviews ({percent:.1f}%)")

print("\n⭐ SENTIMENT BY STAR RATING:")
sentiment_by_star = pd.crosstab(df_merged['stars_review'], df_merged['sentiment_label'])
print(sentiment_by_star)

print("\n🎭 FAKE REVIEWS BY SENTIMENT:")
fake_by_sentiment = df_merged[df_merged['is_fake'] == 1]['sentiment_label'].value_counts()
total_fake = fake_by_sentiment.sum()
for sentiment in ['positive', 'negative', 'neutral']:
    count = fake_by_sentiment.get(sentiment, 0)
    percent = (count / total_fake * 100) if total_fake > 0 else 0
    print(f"    {sentiment.upper():<9}: {count} reviews ({percent:.1f}%)")

print("\n⭐ FAKE REVIEWS BY STAR RATING:")
fake_by_star = df_merged[df_merged['is_fake'] == 1]['stars_review'].value_counts().sort_index()
for star, count in fake_by_star.items():
    percent = (count / total_fake * 100) if total_fake > 0 else 0
    print(f"    {star:.1f} stars: {count} reviews ({percent:.1f}%)")

AFTER CHECK:
📊 SENTIMENT ANALYSIS RESULTS:
   POSITIVE : 6263 reviews (81.7%)
   NEGATIVE : 1069 reviews (13.9%)
   NEUTRAL  : 338 reviews (4.4%)

⭐ SENTIMENT BY STAR RATING:
sentiment_label  negative  neutral  positive
stars_review                                
1                     271        1         0
2                     439       10         0
3                     358      318       293
4                       0        1      2334
5                       1        8      3636

🎭 FAKE REVIEWS BY SENTIMENT:
    POSITIVE : 643 reviews (74.9%)
    NEGATIVE : 183 reviews (21.3%)
    NEUTRAL  : 33 reviews (3.8%)

⭐ FAKE REVIEWS BY STAR RATING:
    1.0 stars: 81 reviews (9.4%)
    2.0 stars: 48 reviews (5.6%)
    3.0 stars: 128 reviews (14.9%)
    4.0 stars: 195 reviews (22.7%)
    5.0 stars: 407 reviews (47.4%)


In [77]:
df_merged['is_fake'].value_counts()

is_fake
0    6811
1     859
Name: count, dtype: int64

```
BEFORE CHECK:
📊 SENTIMENT ANALYSIS RESULTS:
==================================================
   POSITIVE: 6406 reviews (83.5%)
   NEGATIVE: 722 reviews (9.4%)
   NEUTRAL: 545 reviews (7.1%)

⭐ SENTIMENT BY STAR RATING:
sentiment_label  negative  neutral  positive
stars_review                                
1.0                   273        0         0
2.0                   449        0         0
3.0                     0      545       424
4.0                     0        0      2337
5.0                     0        0      3645

🎭 FAKE REVIEWS BY SENTIMENT:
    POSITIVE: 846 reviews (73.5%)
    NEGATIVE: 260 reviews (22.6%)
    NEUTRAL: 45 reviews (3.9%)

⭐ FAKE REVIEWS BY STAR RATING:
    1.0 stars: 183 reviews (15.9%)
    2.0 stars: 77 reviews (6.7%)
    3.0 stars: 265 reviews (23.0%)
    4.0 stars: 96 reviews (8.3%)
    5.0 stars: 530 reviews (46.0%)
```

## Check Similarity

In [85]:
# --- 1️⃣ Load data ---
df = pd.read_csv("yelp_labeled_recombined.csv")

# Pilih kolom teks
text_col = "normalized_text"  # bisa diganti ke "lemmatized_dl" atau "lemmatized_ml"

# --- 2️⃣ TF-IDF Vectorization ---
print("🔍 Menghitung TF-IDF matrix untuk semua review...")
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
tfidf_matrix = vectorizer.fit_transform(df[text_col])

# --- 3️⃣ Cosine Similarity ---
print("📈 Menghitung cosine similarity antar review...")
similarity_matrix = cosine_similarity(tfidf_matrix)

# --- 4️⃣ Temukan similarity tertinggi untuk setiap review ---
print("🔎 Menentukan review paling mirip untuk setiap baris...")

max_similarities = []
most_similar_ids = []

for i in range(similarity_matrix.shape[0]):
    # Set similarity ke diri sendiri jadi 0 biar gak ikut dihitung
    sims = similarity_matrix[i].copy()
    sims[i] = 0
    j = np.argmax(sims)
    max_similarities.append(sims[j])
    most_similar_ids.append(df.loc[j, "review_id"])

# Tambahkan kolom baru ke DataFrame asli
df["max_similarity_score"] = max_similarities
df["most_similar_review_id"] = most_similar_ids

print("✅ Kolom similarity berhasil ditambahkan ke dataset!")

# --- 5️⃣ Simpan hasil ---
df.to_csv("yelp_labeled_with_similarity.csv", index=False)
print("💾 Dataset disimpan sebagai: yelp_labeled_with_similarity.csv")

🔍 Menghitung TF-IDF matrix untuk semua review...
📈 Menghitung cosine similarity antar review...
🔎 Menentukan review paling mirip untuk setiap baris...
✅ Kolom similarity berhasil ditambahkan ke dataset!
💾 Dataset disimpan sebagai: yelp_labeled_with_similarity.csv


In [86]:
# --- 6️⃣ Cek review dengan similarity tinggi ---
threshold = 0.80
high_sim_df = df[df["max_similarity_score"] >= threshold].copy()

print(f"\n📊 Ditemukan {len(high_sim_df)} review dengan similarity >= {threshold}\n")
print("📝 Menampilkan pasangan review yang dianggap mirip:\n")

# Buat index mapping biar bisa cari baris keberapa berdasarkan review_id
id_to_index = {rid: idx for idx, rid in enumerate(df["review_id"])}

# Loop setiap baris dan tampilkan pasangan review
for idx, row in high_sim_df.iterrows():
    review_id = row["review_id"]
    most_similar_id = row["most_similar_review_id"]
    similarity = row["max_similarity_score"]

    # Ambil teks review asli dan review yang mirip
    text_1 = row[text_col]
    text_2 = df.loc[df["review_id"] == most_similar_id, text_col].values
    text_2 = text_2[0] if len(text_2) > 0 else "(review tidak ditemukan)"

    # Dapatkan nomor baris (index dalam df)
    index_1 = id_to_index.get(review_id, "N/A")
    index_2 = id_to_index.get(most_similar_id, "N/A")

    # Tampilkan hasil dengan format yang rapi
    print(f"🆔 Review ID: {review_id} (baris ke-{index_1})")
    print(f"💬 Review Text:\n{text_1}\n")
    print(f"🔥 Most Similar Review ID: {most_similar_id} (baris ke-{index_2})")
    print(f"💬 Similar Review:\n{text_2}\n")
    print(f"📈 Similarity Score: {similarity:.4f}")
    print("─" * 100)

# --- Simpan hasil ke file ---
merged = high_sim_df.copy()
merged["index_1"] = merged["review_id"].map(id_to_index)
merged["index_2"] = merged["most_similar_review_id"].map(id_to_index)
merged["most_similar_text"] = merged["most_similar_review_id"].map(
    df.set_index("review_id")[text_col]
)
merged.to_csv("high_similarity_pairs.csv", index=False)

print("\n💾 File lengkap dengan pasangan review disimpan sebagai: high_similarity_pairs.csv")


📊 Ditemukan 68 review dengan similarity >= 0.8

📝 Menampilkan pasangan review yang dianggap mirip:

🆔 Review ID: 3lhIh6dIAVSh2ya7hOEkpQ (baris ke-33)
💬 Review Text:
You know food is good when after eating it you cannot stop licking the roof of your mouth to get the flavor just one more time nom nom nom nom The line was long but it was well worth the wait and moved quicker than you would think Oh my chargrilled oysters I wanted to suck the she will dry We went back for seconds with the fresh also yummy The gumbo was a solid choice as well And then finally for the bread pudding holy delicious nom nom nom nom Service was good and the food came out quickly They had a pretty well stocked bar and some yummy beers on tap This place is a must in NOLA nom nom

🔥 Most Similar Review ID: ASmn8yPIwdETO_hEMbaMvA (baris ke-488)
💬 Similar Review:
hi tasty raw oysters and po boys i will gobble you up nom nom nom

📈 Similarity Score: 0.8294
─────────────────────────────────────────────────────────────

In [87]:
# --- 7️⃣ Tandai review dengan similarity tinggi sebagai fake ---
threshold = 0.80  # pastikan sama dengan threshold sebelumnya

# Pastikan kolom is_fake sudah ada, kalau belum, buat kolom dengan default 0
if "is_fake" not in df.columns:
    df["is_fake"] = 0

# Hitung jumlah baris sebelum diubah
before_count = df["is_fake"].sum()

# Ubah nilai jadi 1 untuk review dengan max_similarity_score >= threshold
mask = df["max_similarity_score"] >= threshold
df.loc[mask, "is_fake"] = 1

# Hitung jumlah baris setelah diubah
after_count = df["is_fake"].sum()

# Hitung berapa yang baru ditandai
changed_count = after_count - before_count

print(f"✅ Semua review dengan similarity >= {threshold} telah diberi label is_fake = 1")
print(f"📊 Jumlah baris yang diubah: {changed_count}")
print(f"📈 Total review dengan label is_fake = 1 sekarang: {after_count}")

# --- Simpan hasil baru ---
df.to_csv("yelp_labeled_with_similarity_and_fake.csv", index=False)
print("💾 Dataset disimpan sebagai: yelp_labeled_with_similarity_and_fake.csv")

✅ Semua review dengan similarity >= 0.8 telah diberi label is_fake = 1
📊 Jumlah baris yang diubah: 40
📈 Total review dengan label is_fake = 1 sekarang: 899
💾 Dataset disimpan sebagai: yelp_labeled_with_similarity_and_fake.csv


In [88]:
df.iloc[5708][['normalized_text','max_similarity_score', 'is_fake']]

normalized_text         Definitely a star I was solo and sat at the bar ordered dz raw an oyster shot and an acme sunset Everything was delicious The oysters in the shot was done with vodka I have had it done with beer usually left a fresh good taste in my mouth when it went down Every oyster was shucked on the spot and fresh they were big and juicy The guys that was shucking them were super friendly They let many customers go Behind the bar to take pictures together All the employees were joking around very casually and everyone look like they got along well and was not miserable and hated each other like every where else The bill was only for all that did I mention the guys behind the bar was so cool he there in half a dz for free He said thanks for being patient and he apologized that I had to wait a few minutes They are too kind I did not wait long at all and it was when they had people line up outside down a whole block So yeah customer service was on point I hear their other sele

In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7670 entries, 0 to 7669
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   business_name              7670 non-null   object 
 1   review_id                  7670 non-null   object 
 2   user_id                    7670 non-null   object 
 3   date                       7670 non-null   object 
 4   stars_review               7670 non-null   int64  
 5   stars_business_average     7670 non-null   int64  
 6   normalized_text            7670 non-null   object 
 7   lemmatized_ml              7670 non-null   object 
 8   lemmatized_dl              7670 non-null   object 
 9   lemmatized_bert            7670 non-null   object 
 10  sentiment_label            7670 non-null   object 
 11  daily_spike_score          7670 non-null   float64
 12  hourly_spike_score         7670 non-null   float64
 13  temporal_spike_score       7670 non-null   float

In [90]:
sentiment_counts = df['sentiment_label'].value_counts()
sentiment_counts

total_reviews = len(df)
total_reviews

print("AFTER UBAH:")
print("📊 SENTIMENT ANALYSIS RESULTS:")
print("=" * 50)

for sentiment in ['positive', 'negative', 'neutral']:
    count = sentiment_counts.get(sentiment, 0)
    percent = (count / total_reviews) * 100
    print(f"   {sentiment.upper():<9}: {count} reviews ({percent:.1f}%)")

print("\n⭐ SENTIMENT BY STAR RATING:")
sentiment_by_star = pd.crosstab(df['stars_review'], df['sentiment_label'])
print(sentiment_by_star)

print("\n🎭 FAKE REVIEWS BY SENTIMENT:")
fake_by_sentiment = df[df['is_fake'] == 1]['sentiment_label'].value_counts()
total_fake = fake_by_sentiment.sum()
for sentiment in ['positive', 'negative', 'neutral']:
    count = fake_by_sentiment.get(sentiment, 0)
    percent = (count / total_fake * 100) if total_fake > 0 else 0
    print(f"    {sentiment.upper():<9}: {count} reviews ({percent:.1f}%)")

print("\n⭐ FAKE REVIEWS BY STAR RATING:")
fake_by_star = df[df['is_fake'] == 1]['stars_review'].value_counts().sort_index()
for star, count in fake_by_star.items():
    percent = (count / total_fake * 100) if total_fake > 0 else 0
    print(f"    {star:.1f} stars: {count} reviews ({percent:.1f}%)")

AFTER UBAH:
📊 SENTIMENT ANALYSIS RESULTS:
   POSITIVE : 6263 reviews (81.7%)
   NEGATIVE : 1069 reviews (13.9%)
   NEUTRAL  : 338 reviews (4.4%)

⭐ SENTIMENT BY STAR RATING:
sentiment_label  negative  neutral  positive
stars_review                                
1                     271        1         0
2                     439       10         0
3                     358      318       293
4                       0        1      2334
5                       1        8      3636

🎭 FAKE REVIEWS BY SENTIMENT:
    POSITIVE : 679 reviews (75.5%)
    NEGATIVE : 184 reviews (20.5%)
    NEUTRAL  : 36 reviews (4.0%)

⭐ FAKE REVIEWS BY STAR RATING:
    1.0 stars: 81 reviews (9.0%)
    2.0 stars: 50 reviews (5.6%)
    3.0 stars: 130 reviews (14.5%)
    4.0 stars: 213 reviews (23.7%)
    5.0 stars: 425 reviews (47.3%)


In [91]:
df['is_fake'].value_counts()

is_fake
0    6771
1     899
Name: count, dtype: int64

```
AFTER CHECK:
📊 SENTIMENT ANALYSIS RESULTS:
==================================================
   POSITIVE : 6263 reviews (81.7%)
   NEGATIVE : 1069 reviews (13.9%)
   NEUTRAL  : 338 reviews (4.4%)

⭐ SENTIMENT BY STAR RATING:
sentiment_label  negative  neutral  positive
stars_review                                
1                     271        1         0
2                     439       10         0
3                     358      318       293
4                       0        1      2334
5                       1        8      3636

🎭 FAKE REVIEWS BY SENTIMENT:
    POSITIVE : 642 reviews (74.8%)
    NEGATIVE : 183 reviews (21.3%)
    NEUTRAL  : 33 reviews (3.8%)

⭐ FAKE REVIEWS BY STAR RATING:
    1.0 stars: 81 reviews (9.4%)
    2.0 stars: 48 reviews (5.6%)
    3.0 stars: 128 reviews (14.9%)
    4.0 stars: 194 reviews (22.6%)
    5.0 stars: 407 reviews (47.4%)

Setelah Check Label:
is_fake
0    6811
1     859
Name: count, dtype: int64

```